# Análise comparativa de modelos

In [1]:
from IPython.display import Image, display, Markdown
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import joblib

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## 1. Obtenção de Dados

Nessa etapa obtemos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pré-processamento.

In [2]:
df = pd.read_csv("../data/raw/diabetes_prediction_dataset.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,gender,Indica o genêro do paciente,qualitativa,nominal
1,age,Indica a idade do paciente,quantitativa,discreta
2,hypertension,Indica se o paciente possui hipertensão,quantitativa,discreta
3,heart_disease,Indica se o pacinete possui doença no coração,quantitativa,discreta
4,smoking_history,Indica informações sobre o historico de fumant...,qualitativa,nominal
5,bmi,Indica o valor do IMC(Indice de Massa Corporal...,quantitativa,contínua
6,HbA1c_level,Indica o nivel de açucar presente no sangue do...,quantitativa,contínua
7,blood_glucose_level,Indica o nivel de glicose presente no sangue d...,quantitativa,discreta
8,diabetes,Indica se o paciente possui diabetes,quantitativa,discreta


## 2. Preparação de dados

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados.

### Tratamento de dados faltantes:

In [3]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

A partir da informação dada pela saída do código acima, é possível perceber que não existem dados faltantes na tabela.

### Tratamento de dados discrepantes:

In [4]:
# Definindo a variável alvo e as colunas nominais, contínuas e discretas
target_column = 'diabetes'  # A variável alvo

# Filtrando as colunas nominais e contínuas
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal'")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'contínua'")
    .variavel
    .to_list()
)

# Filtrando as colunas discretas
discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'and variavel != @target_column")
    .variavel
    .to_list()
)

# Separando as features (X) da variável alvo (y)
X = df.drop(columns=[target_column], axis=1)
y = df[target_column]
nominal_columns, continuous_columns, discrete_columns

(['gender', 'smoking_history'],
 ['bmi', 'HbA1c_level'],
 ['age', 'hypertension', 'heart_disease', 'blood_glucose_level'])

### 2.1 Pré-processamento

In [5]:
# Pré-processador para variáveis nominais
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')),  # Codificação de variáveis nominais
])

# Pré-processador para variáveis contínuas
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Tratamento de dados faltantes
    ('normalization', MinMaxScaler())  # Normalização de dados contínuos
])

# Pré-processador para variáveis discretas
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('normalization', MinMaxScaler())  # Normalização de variáveis discretas
])

# Combinando os pré-processadores usando ColumnTransformer
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])

model = LogisticRegression()

In [6]:
# Aplicando o pré-processamento ao conjunto de dados
preprocessor.fit_transform(X)
X_transform = preprocessor.transform(X)
X_transform

array([[0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.27272727],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.35454545],
       ...,
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.34090909],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.09090909],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.04545455]])

## 3. Seleção de modelos

### 3.1. Metodologia

Iremos análisar quatro modelos, que serão testados utilizando um método de validação, a saber:

- Complement Naive Bayes
- Decision Tree
- Random Forest
- Logistic Regression 

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

- **Acurácia (accuracy)**: proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados;
- **Precisão (precision)**: proporção entre dados corretamente previstos como positivos e o total de observações positivas.
- **Recall**: proporção entre dados corretamente previstos como positivos com o total de observações.
- **F1-score**: média entre precision e recall, portanto levando em conta tanto falsos positivos quanto falsos negativos.

### 3.2. Configuração do experimento

In [7]:
# Configurações do experimento
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = 0.3
random_state = 42
scoring = 'accuracy'

# Métricas para análises
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Configurações do modelo
max_iter = 1000
models = [
    ('Complement Naive Bayes', ComplementNB(), {'norm': [True, False]}),
    ('Decision Tree',  DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
    ('Logistic Regression', LogisticRegression(), {'C' : np.logspace(-4, 4, 20)}),
]

In [8]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=2,
        cv=cross_validate_grid_search
    )
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=2,
        scoring=metrics
    )
    scores['model_name'] = [model_name] * n_splits_comparative_analysis
    df_scores = pd.DataFrame(scores)
    df_scores =  df_scores.drop(columns=['model_name'])
    df_scores = df_scores.agg(['mean', 'std'])
    display(df_scores)
    results = pd.concat([results, pd.DataFrame(scores)], ignore_index=True)

running Complement Naive Bayes...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,0.250388,0.067390,0.721627,0.544598,0.614368,0.529392
std,0.030550,0.011869,0.009641,0.001350,0.003011,0.004527


running Decision Tree...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,1.581623,0.049074,0.971580,0.981202,0.835390,0.892688
std,0.092920,0.006744,0.000424,0.004642,0.003491,0.001856


running Random Forest...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,15.974738,0.076905,0.971673,0.984873,0.833528,0.892481
std,0.385394,0.021456,0.000442,0.000301,0.002173,0.001682


running Logistic Regression...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,13.828449,0.062725,0.960010,0.918722,0.806368,0.851915
std,1.359988,0.006603,0.000602,0.003268,0.004927,0.003180


In [9]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("### 3.3 Resultados gerais e discussão"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 3.3 Resultados gerais e discussão

Como pode ser visto, o classificador Decision Tree obteve melhores resultados para a maioria das métricas, portanto, podemos obter os melhores parâmetros deste modelo e salvá-lo em disco para utilização em uma próxima etapa.

In [10]:
#Obtem o modelo e os parametros ganhadores
model_name, model_object, model_parameters  = [foo for foo in models if foo[0] == "Decision Tree"][0] 


model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=2,
        cv=cross_validate_grid_search
    )

approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model_grid_search)
])

approach.fit(X, y) #Seleciona o approach

print(f"Hiper parâmetros do modelo: {approach.steps[1][1].best_params_}")

Hiper parâmetros do modelo: {'criterion': 'gini', 'max_depth': 8}


In [11]:
joblib.dump(approach, '../models/model.joblib') # Salva o modelo em disco

['../models/model.joblib']